# Kerchunk GRIDMET data
Using NetCDF files on s3

In [ ]:
import numpy as np
import xarray as xr
import fsspec
from pathlib import Path
import ujson
import os

In [ ]:
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr

In [ ]:
#url = '/caldera/projects/usgs/water/wbeep/onhm_dev/climate/gridmet_raw'
fs = fsspec.filesystem('s3', profile='esip-qhub')

In [ ]:
%%time
#flist = fs_local.glob('/caldera/projects/usgs/water/wbeep/onhm_dev/climate/gridmet_raw_rechunk/*_nc4.nc')
flist = sorted(fs.glob('s3://esip-qhub/usgs/nhgf/gridmet_100mb/*.nc'))
flist

In [ ]:
json_dir = 's3://esip-qhub/usgs/nhgf/gridmet_100mb/jsons/'

In [ ]:
so = dict(mode='rb', profile='esip-qhub', default_fill_cache=False, default_cache_type='first')

In [ ]:
def gen_json(u):
    with fs.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)
        fstem = Path(u).stem 
        outf = f'{json_dir}{fstem}.json'
        print(outf)
        with fs.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

#### Create the Kerchunk JSON for each individual file

In [ ]:
%%time
for f in flist:
    gen_json(f)

#### Check one to see if they look okay

In [ ]:
rpath = 's3://esip-qhub/usgs/nhgf/gridmet/jsons/gridmet_ppt_1979-2020_nc4.json'
s_opts = {'requester_pays':True, 'skip_instance_cache':True}
r_opts = {'requester_pays':True}
fs2 = fsspec.filesystem("reference", fo=rpath, ref_storage_args=s_opts,
                       remote_protocol='s3', remote_options=r_opts)

In [ ]:
fs2.ls('')

In [ ]:
m = fs2.get_mapper("")
ds = xr.open_dataset(m, engine="zarr",chunks={'day': 1220, 'lat': 98, 'lon': 231},
                    backend_kwargs={'consolidated':False})

In [ ]:
ds